# Google Play and App Store Data Analysis
    
This project was designed to analyze data from android's google play and apple's app store to potentially recommend a genre for an app to developers that could be profitable for both stores. We are analyzing free english apps to better understand which categories are most popular. 

In [86]:
from csv import reader

opened_file = open('googleplaystore2018.csv', encoding='utf8')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

opened_file = open('applestore2017.csv', encoding='utf8')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

Designing code to filter and visualize application data within the Google Play Store and App Store.

In [88]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') 

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
##Exploring data sets       
print(android_header)
print('\n')
explore_data(android, 0, 3, True)
print('\n')
print(ios_header)
print('\n')
explore_data(ios, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', '15-Jan-18', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '38

# Cleaning up the data
## Deleting Wrong and Inaccurate Data
A particular row was not aligned with the dataset within the googleplay csv file and needs to be deleted to not impact results.

In [3]:
print(android[10472])  # incorrect row
print('\n')
print(android_header)  # header
print('\n')
print(android[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', '11-Feb-18', '1.0.19', '4.0 and up', '']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


Here we deleted row 10472 because the row was inaccurate in comparison to remaining rows and had to be deleted to not impact results.

In [4]:
print(len(android))
del android[10472]  # don't run this more than once
print(len(android))

10841
10840


Some applications may have multiple entries due to popularity. We selected Twitter as it is a multifaceted platform.

In [77]:
for app in android:
    name = app[0]
    if name =='Twitter':
        print(app)

['Twitter', 'NEWS_AND_MAGAZINES', '4.3', '11667403', 'Varies with device', '500,000,000+', 'Free', '0', 'Mature 17+', 'News & Magazines', '6-Aug-18', 'Varies with device', 'Varies with device']
['Twitter', 'NEWS_AND_MAGAZINES', '4.3', '11667403', 'Varies with device', '500,000,000+', 'Free', '0', 'Mature 17+', 'News & Magazines', '6-Aug-18', 'Varies with device', 'Varies with device']
['Twitter', 'NEWS_AND_MAGAZINES', '4.3', '11657972', 'Varies with device', '500,000,000+', 'Free', '0', 'Mature 17+', 'News & Magazines', '30-Jul-18', 'Varies with device', 'Varies with device']


## Removing duplicate entries
The criterion we used for removing the duplicate entries from the dataset was after we reviewed the output, which is being varied by review factor. We've seen that the difference between these duplicate values are the number of reviews. This means that the more reviews there are, the more recent the data is. Now we are taking the most recent data and removing the rest of each of the duplicate apps. 

In [68]:
duplicate_apps = []
unique_apps = []

for app in android:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
    
print('Number of duplicate apps:', len(duplicate_apps))
print('Examples of duplicate apps:', duplicate_apps[:15])
print('\n')
print('Number of unique android apps',len(unique_apps))
print('Examples of unique android apps:', unique_apps[:15])

Number of duplicate apps: 1181
Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


Number of unique android apps 9659
Examples of unique android apps: ['Photo Editor & Candy Camera & Grid & ScrapBook', 'Coloring book moana', 'U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'Sketch - Draw & Paint', 'Pixel Draw - Number Art Coloring Book', 'Paper flowers instructions', 'Smoke Effect Photo Maker - Smoke Editor', 'Infinite Painter', 'Garden Coloring Book', 'Kids Paint Free - Drawing Fun', 'Text on Photo - Fonteee', 'Name Art Photo Editor - Focus n Filters', 'Tattoo Name On My Photo Editor', 'Mandala Coloring Book', '3D Color Pixel by Number - Sandbox Art Coloring']


Here we'll create the dictionary reviews_max consisting of unique apps names and their respective maximum reviews.

In [69]:
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews
        
print('Expected length:', len(android) - 1181)
print('\n')
print('Actual length:', len(reviews_max))

Expected length: 9659


Actual length: 9659


After ensuring our reviews_max dictionary is functioning properly, we now use it to subsequently develop a new dataset android_clean that consists solely of unique apps with their highest review.

In [75]:
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if (n_reviews == reviews_max[name]) and (name not in already_added):
        android_clean.append(app)
        already_added.append(name) 

explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', '8-Jun-18', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


We've used the reviews_max dictionary to formulate a new list without duplicates.

In [13]:
print(ios[813][1])
print(ios[6731][1])
print('\n')
print(android_clean[4412][0])
print(android_clean[7940][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜


中国語 AQリスニング
لعبة تقدر تربح DZ


## Removing non-English Apps
Here we have detected non English app names and will subsequently formulate functions to filter out such apps as our prospective audience is English speaking. We use ord() for characters that may not be recognized within the English language such as emojis, as they have corresponding numerical values which could validate them rather than exclude them for our data.

In [65]:
def is_english(string):
    for character in string:
        if ord(character) > 127:
            return False
    return True

print(is_english('Twitter'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(ord('™'))
print(ord('😜'))

True
False
False
False
8482
128540


Notice how two English apps have been deemed False due to special characters not falling within the 0-127 ASCII table. To better sort through these English apps as to not exclude them we will be adding conditional statements with values that collectively should assure they populate.

In [66]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

print(is_english('Twitter'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))
print(ord('™'))
print(ord('😜'))

True
False
True
True
8482
128540


Following the addition of conditional statements, Instachat and Docs To Go are now qualifying is True English apps.

In [67]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)
        
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', '7-Jan-18', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', '1-Aug-18', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', '8-Jun-18', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


['284882215', 'Facebook', '389879808', 'USD', '0', '2974676', '212', '3.5', '3.5', '95', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0', '2161558', '1289', '4.5', '4', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '

After exploring and filtering through Google Play and App Store datasets we have a total of 9,614 non-English apps in Google Play and 6,184 non-English apps in the App Store.

## Isolating Free Apps
Currently our data is reflecting a combination of free and paid for applications within Google Play and the App Store. Our goal is to analyze free applications exclusively. To do so, we are compiling conditional statements to filter out all applications that are not free or price valued at $0 to users.

In [45]:
android_final = []
ios_final = []

for app in android_english:
    name = app[0]
    price = app[7]
    if price == '0':
        android_final.append(app)
        
for app in ios_english:
    name = app[0]
    price = app[4]
    if price == '0':
        ios_final.append(app)
        
print(len(android_final))
print(len(ios_final))

8864
3222


Now we have 8,864 applications remaining in the Google Play Store that we know to be free. Meanwhile, the App Store had a reduction of just under 50% with only 3,222 free applications in total.

## Analyzing and Reviewing Data
Our aim is to determine the kinds of free apps that are likely to attract more users because our revenue is highly influenced by the number of users. We'll be assessing a variety of categories to build a beta Android version of an app. If the app tests well with users in Google Play, proving it can be successful and therefore profitable, we can assume it will in the App Store as well. 

We'll begin this analysis by testing the most popular genres in both markets. For this, we'll build a frequency table for the prime_genre column of the App Store dataset and the Genres and Category columns of the Googly Play dataset.

In [46]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages


def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [50]:
display_table(ios_final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


The prime_genre column of the App Store dataset as reflected above indicates that Games is the most popular genre occupying 58.16% of the market, followed by Entertainment 7.88% and Photo & Video 4.96% respectively. Meanwhile Navigation at 0.18%, Medical at 0.18% and Catalogs at 0.12% are the least popular within the market.

In [22]:
display_table(android_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

The Genre column of the Google Play Store dataset as reflected above indicates that Family is the most popular genre is the most popular genre occupying 18.9% of the market, followed by Game at 9.72% and Tools at 8.46%. On the other end, Art_and_Design occupies 0.64%, Comics 0.62% and Beauty 0.59% of the market.

In [23]:
display_table(android_final, -4)

Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

The Category column of the Google Play Store dataset as reflected above indicates that Tools is the most popular category occupying 8.44% of the market, followed by Entertainment 6.06% and Education 5.34% respectively. Meanwhile Art & Design;Action & Adventure at 0.011%, Arcade;Pretend Play 0.011% and Adventure;Education at 0.011% are the least popular within the market.

## Most Popular IOS Apps by Genre
We were able to determine through the freq_table dictionary and display_table function what the popular genres within the prime_genre column of the App Store dataset were. Now we are looking to determine what genres are the most popular by users rather than number of apps within the market. To accomplish this we must calculate the average number of user ratings per genre in the App Store.

In [48]:
genres_ios = freq_table(ios_final, -5)

for genre in genres_ios:
    total = 0
    len_genre = 0
    for app in ios_final:
        genre_app = app[-5]
        if genre_app == genre:            
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
    avg_n_ratings = total / len_genre
    print(genre, ':', avg_n_ratings)

Social Networking : 71548.34905660378
Photo & Video : 28441.54375
Games : 22788.6696905016
Music : 57326.530303030304
Reference : 74942.11111111111
Health & Fitness : 23298.015384615384
Weather : 52279.892857142855
Utilities : 18684.456790123455
Travel : 28243.8
Shopping : 26919.690476190477
News : 21248.023255813954
Navigation : 86090.33333333333
Lifestyle : 16485.764705882353
Entertainment : 14029.830708661417
Food & Drink : 33333.92307692308
Sports : 23008.898550724636
Book : 39758.5
Finance : 31467.944444444445
Education : 7003.983050847458
Productivity : 21028.410714285714
Business : 7491.117647058823
Catalogs : 4004.0
Medical : 612.0


In an unexpected turnaround, Navigation which only had 0.18% of applications within the App Store market, is first in popularity with an average of 86,090 ratings as it is widely used among users.

In [49]:
for app in ios_final:
    if app[-5] == 'Navigation':
        print(app[1], ':', app[5]) # print name and number of ratings

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


As seen above, popularity within the Navigation genre could be deceptive as Waze - GPS Navigation, Maps & Real-time Traffic and Google Maps dominate the genre, leaving little room for developing competitive applications.

In [93]:
for app in ios_final:
    if app[-5] == 'Reference':
        print(app[1], ':', app[5])

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826
WWDC : 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718
VPN Express : 14
Real Bike Traffic Rider Virtual Reality Glasses : 8
教えて!goo : 0
Jishokun-Japanese English Dictionary & Translator : 0


The Reference genre was indicative of a similar conclusion. Runner-up to Navigation, it too has a notable user base for Bible and Dictionary.com Dictionary & Thesaurus in comparison to other apps. The difference however, is that there is an increased selection and potential for growth through sub-genres or additional features. Perhaps developing a journal app would prove popular among users with an option to share with others. Document your thoughts and life journey as and maintain it as a reference for yourself and possibly others later in life.

In [98]:
for app in ios_final:
    if app[-5] == 'News':
        print(app[1], ':', app[5])

Twitter : 354058
Fox News : 132703
CNN: Breaking US & World News, Live Video : 112886
Reddit Official App: All That's Trending and Viral : 67560
USA TODAY : 61724
ABC News - US & World News + Live Video : 48407
NBC News : 32881
HuffPost - News, Politics & Entertainment : 29107
The Washington Post Classic : 18572
WIRED Magazine : 12074
CBS News - Watch Free Live Breaking News : 11691
The Guardian : 8176
AOL: News, Email, Weather & Video : 5233
SmartNews - Trending News & Stories : 4645
MSNBC : 3692
LotteryHUB : 2417
theSkimm : 1765
Quartz • News in a whole new way : 1267
Lotto Results - Mega Millions Powerball Lottery : 794
TopBuzz: Best Viral Videos, GIFs, TV & News : 692
Ticket Scanner for Powerball & MegaMillions Pool : 581
FOCUS Online - Aktuelle Nachrichten : 373
SPIEGEL ONLINE - Nachrichten : 299
n-tv Nachrichten : 273
CNN Politics : 254
Tagesschau : 233
Fresco — Be a part of the news : 219
News Break - Local & World Breaking News & Radio : 173
OPM Alert : 172
franceinfo - l'actua

News held 1.33% of the overall app market within the App Store yet 21,248 average number or ratings. We were surprised to find the Twitter app within this catergory, and dominating it at that with 354,058 ratings. We were expecting to find this app within the Social Networking genre.

## Most Popular Android Apps by Genre
The Google Play App Store fortunately has an "Installs" column to track their most downloaded apps.


In [81]:
display_table(android_final, 5)

1,000,000+ : 15.726534296028879
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.198555956678701
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.7721119133574
5,000+ : 4.512635379061372
10+ : 3.5424187725631766
500+ : 3.2490974729241873
50,000,000+ : 2.3014440433213
100,000,000+ : 2.1322202166064983
50+ : 1.917870036101083
5+ : 0.78971119133574
1+ : 0.5076714801444043
500,000,000+ : 0.2707581227436823
1,000,000,000+ : 0.22563176895306858
0+ : 0.04512635379061372
0 : 0.01128158844765343


As you can tell from this analysis, the data isn't quite too precise. We'll leave n_installs to help substitute additional characters with space. These characters would be commas and addition signs, so that we may covert our our n_istalls to float. Through this loop we will calculate the average number of installs for each genre.

In [99]:
categories_android = freq_table(android_final, 1)

for category in categories_android:
    total = 0
    len_category = 0
    for app in android_final:
        category_app = app[1]
        if category_app == category:            
            n_installs = app[5]
            n_installs = n_installs.replace(',', '')
            n_installs = n_installs.replace('+', '')
            total += float(n_installs)
            len_category += 1
    avg_n_installs = total / len_category
    print(category, ':', avg_n_installs)

ART_AND_DESIGN : 1986335.0877192982
AUTO_AND_VEHICLES : 647317.8170731707
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
COMICS : 817657.2727272727
COMMUNICATION : 38456119.167247385
DATING : 854028.8303030303
EDUCATION : 1833495.145631068
ENTERTAINMENT : 11640705.88235294
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
HOUSE_AND_HOME : 1331540.5616438356
LIBRARIES_AND_DEMO : 638503.734939759
LIFESTYLE : 1437816.2687861272
GAME : 15588015.603248259
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
TRAVEL_AND_LOCAL : 13984077.710144928
TOOLS : 10801391.298666667
PERSONALIZATION : 5201482.6122448975
PRODUCTIVITY : 16787331.344927534
PARENTING : 542603.6206896552
WEATHER : 5074486.197183099
VIDEO_PLAYERS : 24727872.452830188
NEWS_AND_

Here we see that the most popular installs are Communication followed by Video Player. These categories tend to be broad and sometimes have applications that may not seem as though they fall within these categories. For example we have Whatsapp, Android messengers and Skype under Communication. Meanwhile, Video Players has Google Play and Youtube. The same happens within other genres such as Productivity where Tools is quite popular but with a vast and overwhelmingly competitive market.

In [54]:
for app in android_final:
    if app[1] == 'COMMUNICATION' and (app[5] == '1,000,000,000+'
                                      or app[5] == '500,000,000+'
                                      or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
imo beta free calls and text : 100,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Skype - free IM & video calls : 1,000,000,000+
Who : 100,000,000+
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000+
LINE: Free Calls & Messages : 500,000,000+
Google Chrome: Fast & Secure : 1,000,000,000+
Firefox Browser fast & private : 100,000,000+
UC Browser - Fast Download Private & Secure : 500,000,000+
Gmail : 1,000,000,000+
Hangouts : 1,000,000,000+
Messenger Lite: Free Calls & Messages : 100,000,000+
Kik : 100,000,000+
KakaoTalk: Free Calls & Text : 100,000,000+
Opera Mini - fast web browser : 100,000,000+
Opera Browser: Fast and Secure : 100,000,000+
Telegram : 100,000,000+
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000+
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000+
Viber Mess

In [55]:
under_100_m = []

for app in android_final:
    n_installs = app[5]
    n_installs = n_installs.replace(',', '')
    n_installs = n_installs.replace('+', '')
    if (app[1] == 'COMMUNICATION') and (float(n_installs) < 100000000):
        under_100_m.append(float(n_installs))
        
sum(under_100_m) / len(under_100_m)

3603485.3884615386

A genre that offers popularity with a degree of competition is the "News and Magazine" genre. This genre is not currently dominated by a few applications, providing us potential for growth. This genre matches our findings in the Google Play Store and is ideally what we are seeking.

In [72]:
for app in android_final:
    if app[1] == 'NEWS_AND_MAGAZINES':
        print(app[0], ':', app[5])

Fox News – Breaking News, Live Video & News Alerts : 10,000,000+
NEW - Read Newspaper, News 24h : 10,000,000+
BaBe + - Indonesian News : 1,000,000+
daily News : 1,000,000+
BBC News : 10,000,000+
Free TV Shows App:News, TV Series, Episode, Movies : 1,000,000+
News24 : 1,000,000+
Le Monde, the continuous news : 5,000,000+
Wireless news : 1,000,000+
CNN Breaking US & World News : 10,000,000+
Gnoche entertainment news · sports news is also free : 5,000,000+
BaBe - Read News : 10,000,000+
Nigeria News NAIJ.com : 1,000,000+
detikcom - Latest & Most Complete News : 10,000,000+
Dailyhunt (Newshunt) - Latest News, Viral Videos : 50,000,000+
BaBe Lite - Read Quota Saving News : 1,000,000+
ARY NEWS URDU : 500,000+
Bengali Newspaper - The first L. : 1,000,000+
Read- Latest News, Information, Gossip and Politics : 10,000,000+
Reddit: Social News, Trending Memes & Funny Videos : 10,000,000+
BBC Persian | BBC Farsi News : 100,000+
Opera News - Trending news and videos : 10,000,000+
Topbuzz: Breaking 

The News_And_Magazines genre includes apps like Fox News, Microsoft News, Apple Daily Apple News, Huff Post, AP Mobile, etc. It appears as though there continues to be a small number of massively popular apps skewing the overall average. 

In [73]:
for app in android_final:
    if app[1] == 'NEWS_AND_MAGAZINES' and (app[5] == '1,000,000,000+'
                                            or app[5] == '500,000,000+'
                                            or app[5] == '100,000,000+'):
        print(app[0], ':', app[5])

Twitter : 500,000,000+
Flipboard: News For Our Time : 500,000,000+
Google News : 1,000,000,000+


Only a few apps are exponentially popular so this market still has potential. Lets look at applications we can find in this category that are in the middle range of popularity.

In [100]:
for app in android_final:
    if app[1] == 'NEWS_AND_MAGAZINES' and (app[5] == '1,000,000+'
                                            or app[5] == '5,000,000+'
                                            or app[5] == '10,000,000+'
                                            or app[5] == '50,000,000+'):
        print(app[0], ':', app[5])

Fox News – Breaking News, Live Video & News Alerts : 10,000,000+
NEW - Read Newspaper, News 24h : 10,000,000+
BaBe + - Indonesian News : 1,000,000+
daily News : 1,000,000+
BBC News : 10,000,000+
Free TV Shows App:News, TV Series, Episode, Movies : 1,000,000+
News24 : 1,000,000+
Le Monde, the continuous news : 5,000,000+
Wireless news : 1,000,000+
CNN Breaking US & World News : 10,000,000+
Gnoche entertainment news · sports news is also free : 5,000,000+
BaBe - Read News : 10,000,000+
Nigeria News NAIJ.com : 1,000,000+
detikcom - Latest & Most Complete News : 10,000,000+
Dailyhunt (Newshunt) - Latest News, Viral Videos : 50,000,000+
BaBe Lite - Read Quota Saving News : 1,000,000+
Bengali Newspaper - The first L. : 1,000,000+
Read- Latest News, Information, Gossip and Politics : 10,000,000+
Reddit: Social News, Trending Memes & Funny Videos : 10,000,000+
Opera News - Trending news and videos : 10,000,000+
Topbuzz: Breaking News, Videos & Funny GIFs : 10,000,000+
Pulse Nabd - World News, 

## Conclusion
Overall in this project we succeeded in analyzing data from the Google Play Store and App Store with the goal of recommending an app that can be profitable across both markets

We concluded that from compiling a single source from contributions from popular local and national news sources, for ex. The Boston Globe, into a single app could be beneficial and profitable. Users would be able to access all news within a single app. There would be an option within the app for users to share opinions and debate amongst each other on local and national issues. The app could also charge to upgrade for the opportunity to vote from within the app itself for any local issues at hand.